In [3]:
import sys
sys.path.append("/home/juliafalcao")
sys.path.append("/home/juliafalcao/experiments")

from experiments.constants import * 
from experiments.utils import *

import pandas as pd
import json

In [4]:
# SRC, TGT = SPANISH, BASQUE
SRC, TGT = ENGLISH, MALTESE

In [5]:
# batches_filepath = f"/netscratch/falcao/data/{SRC}-{TGT}/eval-set/batches.{SRC}-{TGT}.json"
batches_filepath = f"/home/juliafalcao/thesis_data/{SRC}-{TGT}/batches.{SRC}-{TGT}.json"
with open(batches_filepath, mode="r") as f:
    j = json.load(f)

In [6]:
df_filepath = f"/home/juliafalcao/thesis_data/{SRC}-{TGT}/df_eval.csv"
df_eval = pd.read_csv(df_filepath)

In [7]:
dfs = []
for batch_json in j:
    _df = pd.DataFrame(batch_json["items"])
    _df["_batchNo"] = batch_json["task"]["batchNo"]
    dfs.append(_df)

df = pd.concat(dfs)
df = df[["_batchNo", *df.columns[:-1]]]

In [8]:
print("total items:", len(df))

total items: 1500


In [9]:
ids = df.groupby("itemID").itemType.unique().apply(len).sort_values(ascending=False).where(lambda x: x == 3).dropna().index

In [20]:
df[df.itemID.isin(ids) & (df.itemType == "BAD") & (df.targetID == "gtrans.mt")].sourceText.unique()

array(['Regional and seasonal severe weather phenomena include blizzards, snowstorms, ice storms, and dust storms.',
       'The grandparents enjoyed the social aspects of learning in informal clubs with the other grandparents, and they participated very enthusiastically in the various language competitions, pairing up in a team with their own grandchild.',
       'Supervisors should be adequately trained about risks and control measures connected with the work of young people, and made aware of any restrictions on the tasks young people can do.',
       "He would go on talking to anyone who listened. He just never stopped. Even if you didn't ask for it, he'd let you know exactly how he felt.",
       'While immigration check is usually absent or a formality when you arrive in your homeland, customs control can be a hassle.',
       'Nowadays air travel is only rarely booked directly through the airline without first searching and comparing prices.',
       'There were also toilets in 

In [26]:
s = "Nowadays air travel is only rarely booked directly through the airline without first searching and comparing prices."
df[(df.sourceText == s) & (df.targetID.isin(["gtrans.mt", "ref.mt"]))][["itemType", "targetID", "sourceText", "targetText"]].to_dict(orient="records")

[{'itemType': 'REF',
  'targetID': 'ref.mt',
  'sourceText': 'Nowadays air travel is only rarely booked directly through the airline without first searching and comparing prices.',
  'targetText': 'Illum il-ġurnata l-ivvjaġġar bl-ajru rari jiġi bbukkjat direttament permezz tal-linja tal-ajru mingħajr ma l-ewwel isir tiftix u paragunar tal-prezzijiet.'},
 {'itemType': 'TGT',
  'targetID': 'gtrans.mt',
  'sourceText': 'Nowadays air travel is only rarely booked directly through the airline without first searching and comparing prices.',
  'targetText': 'Illum il-ġurnata l-ivvjaġġar bl-ajru huwa rari biss ibbukkjat direttament permezz tal-linja tal-ajru mingħajr ma l-ewwel wieħed ifittex u jqabbel il-prezzijiet.'},
 {'itemType': 'BAD',
  'targetID': 'gtrans.mt',
  'sourceText': 'Nowadays air travel is only rarely booked directly through the airline without first searching and comparing prices.',
  'targetText': 'Illum il-ġurnata l-ivvjaġġar bl-ajru huwa rari biss ibbukkjat direttament perm

In [25]:
df[(df.targetText.apply(lambda x: "kollok aċċess għall-aħbarijiet riċenti, fatti" in x)) & (df.itemType == "REF")].targetText.to_list()

['Dan is-servizz huwa b’xejn, u b’mod faċli, jista’ jkollok aċċess għall-aħbarijiet riċenti, fatti u figuri, dokumenti legali u għadd kbir ta’ informazzjoni prattika.']

In [45]:
identical_hyps = df[df.targetID.apply(lambda id: "+" in id)]
print("identical hypotheses:", len(identical_hyps))
identical_hyps

identical hypotheses: 0


,_batchNo,_block,_item,itemID,itemType,sourceID,sourceText,targetID,targetText


In [37]:
print("sourceID values:", set(df.sourceID))
assert len(set(df.sourceID)) == 1 # ? not sure

sourceID values: {'src.es'}


In [38]:
print("targetID values:", df.targetID.unique().tolist())

targetID values: ['cmbacktrans.eu', 'euskadi.eu', 'nllb.eu', 'ref.eu', 'cmbacktrans.eu+euskadi.eu']


In [39]:
print("itemType values:", set(df.itemType))
assert set(df.itemType) <= set(["BAD", "CHK", "REF", "TGT"])

print("value counts:")
print(df.itemType.value_counts().to_string())
print()

print("per batch:")
_df = df[df._batchNo == 1]
print(_df.itemType.value_counts().to_string())

itemType values: {'TGT', 'REF', 'BAD'}
value counts:
TGT    1200
BAD     150
REF     150

per batch:
TGT    80
BAD    10
REF    10
